In [4]:
import pandas as pd
pd.set_option("display.max_columns", None)
df = pd.read_csv("data_restaurants_processed3.csv")
df.head()

,name,reviews_general_rating,id,name_ex_extension,name_ex_primary,address_building_id,address_components_0_number,address_components_0_street,address_name,adm_div_1_name,adm_div_2_name,adm_div_3_name,adm_div_4_name,links_entrances_0_geometry_points,point_lat,point_lon,reviews_general_review_count,reviews_general_review_count_with_stars,near_office_1,near_office_2,near_office_3,Average bill,Payment,Cuisine,floor1,floor2,has_lunch,lunch_price_start,has_breakfast,has_reservations,has_takeaway,has_tea_coffee_takeaway,has_wifi,Dishes,has_live_music,has_vip_lounge,has_summer_terrace,has_banquet_halls,Mon_from,Mon_to,Tue_from,Tue_to,Wed_from,Wed_to,Thu_from,Thu_to,Fri_from,Fri_to,Sat_from,Sat_to,Sun_from,Sun_to,office_1_dist,office_1_time,office_2_dist,office_2_time,office_3_dist,office_3_time,Европейская кухня,Паназиатская кухня,Русская кухня,Американская кухня,Грузинская кухня,Постное меню,Вегетарианское меню
0,"Harbor, ресторан",4.6,70000001062703679,ресторан,Harbor,5348660212708350,3,9-я Советская улица,"9-я Советская улица, 3",Санкт-Петербург,Санкт-Петербург,Центральный район,Смольнинское,POINT(30.370701625633792 59.938567148391527),59.938559,30.370694,7.0,9.0,True,True,NaN,1600.0,Наличный расчёт; Оплата картой; Оплата через б...,Американская кухня; Европейская кухня,1.0,1.0,True,400.0,True,True,True,True,True,Паста; Шведский стол; Десерты; Куриные крылышк...,NaN,NaN,NaN,NaN,07:30:00,23:00:00,07:30:00,23:00:00,07:30:00,23:00:00,07:30:00,23:00:00,07:30:00,23:00:00,07:30:00,23:00:00,07:30:00,23:00:00,202.01,2.42,768.95,9.23,4853.62,58.24,True,False,False,True,False,False,False
1,"Kroo cafe, французский ресторан",4.3,5348553838944141,французский ресторан,Kroo cafe,5348660212708114,27,Суворовский проспект,"Суворовский проспект, 27",Санкт-Петербург,Санкт-Петербург,Центральный район,Смольнинское,POINT(30.374200653496967 59.937567679721774),59.937559,30.374205,104.0,160.0,True,True,NaN,3000.0,Наличный расчёт; Оплата картой; Оплата по QR-коду,Европейская кухня; Французская кухня,1.0,1.0,True,850.0,True,True,True,True,True,Паста; Десерты; Кондитерские изделия; Глинтвей...,True,True,True,NaN,08:00:00,23:00:00,08:00:00,23:00:00,08:00:00,23:00:00,08:00:00,23:00:00,08:00:00,00:00:00,09:30:00,00:00:00,10:00:00,23:00:00,397.89,4.77,584.16,7.01,4870.39,58.44,True,False,False,False,False,True,False
2,"Osteria Betulla, остерия",4.3,70000001050842811,остерия,Osteria Betulla,5348660212708348,29,Греческий проспект,"Греческий проспект, 29",Санкт-Петербург,Санкт-Петербург,Центральный район,Смольнинское,POINT(30.370271781770345 59.938672444011928),59.938664,30.370265,167.0,285.0,True,True,NaN,1250.0,Наличный расчёт; Оплата картой; Оплата через б...,Авторская кухня; Итальянская кухня,NaN,NaN,NaN,NaN,NaN,True,True,True,True,Супы; Римская пицца; Паста; Пицца,NaN,NaN,NaN,NaN,12:00:00,23:30:00,12:00:00,23:30:00,12:00:00,23:30:00,12:00:00,23:30:00,12:00:00,23:30:00,12:00:00,23:30:00,12:00:00,23:30:00,184.60,2.22,792.92,9.52,4850.67,58.21,True,False,False,False,False,False,False
3,"Feromon Group, лаундж-бар",4.7,70000001066331086,лаундж-бар,Feromon Group,5348660212794938,4а,8-я Советская улица,"8-я Советская улица, 4а",Санкт-Петербург,Санкт-Петербург,Центральный район,Смольнинское,POINT(30.370589246391752 59.938099263966251),59.938095,30.370605,149.0,186.0,True,True,NaN,1200.0,Наличный расчёт; Оплата картой; Перевод с карт...,Европейская кухня; Паназиатская кухня; Итальян...,2.0,2.0,True,460.0,True,True,True,True,True,Паста; Поке; Глинтвейн; Рис wok; Том-Ям; Суши;...,True,True,True,NaN,10:00:00,06:00:00,10:00:00,06:00:00,10:00:00,06:00:00,10:00:00,06:00:00,10:00:00,06:00:00,10:00:00,06:00:00,10:00:00,06:00:00,250.46,3.01,775.66,9.31,4807.53,57.69,True,True,False,False,False,False,False
4,"Степнов, ресторан",4.6,5348552840330064,ресторан,Степнов,5348660212708111,25,Суворовский проспект,"Суворовский проспект, 25",Санкт-Петербург,Санкт-Петербург,Центральный район,Смольнинское,POINT(30.373745477142503 59.937182542880237),59.937178,30.373761,15.0,20.0,True,True,Na

In [5]:
user_answers = {
    "wanted_cuisines": {
        "Европейская кухня": 0.6, 
        "Паназиатская кухня": 0.8, 
        "Русская кухня": 0.4,
        "Американская кухня": 0.5, 
        "Грузинская кухня": 0.7
    },
    "price_limit": {500: 0.4, 1000: 0.6, 1500: 0.0},
    "food_restrictions": {"Постное меню": 0.3, "Вегетарианское меню": 0.5},
    "walk_time": {5: 0.4, 10: 0.6, 15: 0.2}  # Время пешком
}


In [9]:
# 🔹 Функция для расчета баллов заведений
def calculate_score(row):
    # 1️⃣ Баллы за кухню (one-hot encoding)
    #Берем все кухни, которые выбрали пользователи (wanted_cuisines).
#Проверяем, есть ли эта кухня в заведении (row.get(c, False)).
#Если есть, добавляем ее вес в баллы (get(c, 0)).
#Суммируем баллы за все найденные кухни
    cuisine_score = sum(
        user_answers["wanted_cuisines"].get(c, 0) for c in user_answers["wanted_cuisines"] if row.get(c, False)
    )

    # 2️⃣ Баллы за ограничения по питанию (one-hot encoding)
    #Берем все кухни, которые выбрали пользователи (wanted_cuisines).
    #Проверяем, есть ли эта кухня в заведении (row.get(c, False)).
    #Если есть, добавляем ее вес в баллы (get(c, 0)).
     #Суммируем баллы за все найденные кухни
    restrictions_score = sum(
        user_answers["food_restrictions"].get(r, 0) * 2 for r in user_answers["food_restrictions"] if row.get(r, False)
    )

    # 3️⃣ Баллы за цену 
    price = row["Average bill"]
    if pd.notna(price):  # Проверка, что цена не NaN
        price_score = sum(
            user_answers["price_limit"].get(limit, 0) for limit in user_answers["price_limit"] if price <= limit
        )
    else:
        price_score = 0  # Если нет данных о цене, не учитываем

    # 4️⃣ Баллы за время в пути (берем минимальное из трех офисов)
    walk_time = min(row["office_1_time"], row["office_2_time"], row["office_3_time"])
    if pd.notna(walk_time):  # Проверка, что время не NaN
        walk_score = sum(
            user_answers["walk_time"].get(limit, 0) for limit in user_answers["walk_time"] if walk_time <= limit
        )
    else:
        walk_score = 0  # Если нет данных, не учитываем

    # 5️⃣ Баллы за рейтинг и отзывы
    rating = row["reviews_general_rating"]
    reviews = row["reviews_general_review_count"]
    rating_score = (1 if rating > 4.5 else 0) + (1 if reviews > 200 else 0)

    # 🔹 Общая сумма баллов
    total_score = cuisine_score + restrictions_score + price_score + walk_score + rating_score

    return total_score

# 🔹 Добавляем колонку с баллами
df["total_score"] = df.apply(calculate_score, axis=1)

# 🔹 Сортируем по баллам, затем по рейтингу, затем по количеству отзывов
df_sorted = df.sort_values(by=["total_score", "reviews_general_rating", "reviews_general_review_count"], 
                           ascending=[False, False, False])

# 🔹 Выбираем топ-3 ресторана
top_3_places = df_sorted.head(3)

top_3_places

,name,reviews_general_rating,id,name_ex_extension,name_ex_primary,address_building_id,address_components_0_number,address_components_0_street,address_name,adm_div_1_name,adm_div_2_name,adm_div_3_name,adm_div_4_name,links_entrances_0_geometry_points,point_lat,point_lon,reviews_general_review_count,reviews_general_review_count_with_stars,near_office_1,near_office_2,near_office_3,Average bill,Payment,Cuisine,floor1,floor2,has_lunch,lunch_price_start,has_breakfast,has_reservations,has_takeaway,has_tea_coffee_takeaway,has_wifi,Dishes,has_live_music,has_vip_lounge,has_summer_terrace,has_banquet_halls,Mon_from,Mon_to,Tue_from,Tue_to,Wed_from,Wed_to,Thu_from,Thu_to,Fri_from,Fri_to,Sat_from,Sat_to,Sun_from,Sun_to,office_1_dist,office_1_time,office_2_dist,office_2_time,office_3_dist,office_3_time,Европейская кухня,Паназиатская кухня,Русская кухня,Американская кухня,Грузинская кухня,Постное меню,Вегетарианское меню,total_score
513,"Брат, халяль-кафе",4.7,70000001018353421,халяль-кафе,Брат,5348660212704613,10,Радищева,"Радищева, 10",Санкт-Петербург,Санкт-Петербург,Центральный район,Смольнинское,POINT(30.362448264129416 59.937047852111021),59.937050,30.362431,95.0,112.0,True,True,NaN,500.0,Наличный расчёт; Оплата картой; Перевод с карты,Восточная кухня; Халяльная кухня; Кавказская к...,NaN,NaN,NaN,NaN,NaN,True,True,NaN,True,Лагман; Плов; Манты; Люля-кебаб; Шашлык; Салат...,NaN,NaN,NaN,NaN,11:00:00,23:00:00,11:00:00,23:00:00,11:00:00,23:00:00,11:00:00,23:00:00,11:00:00,23:00:00,11:00:00,23:00:00,11:00:00,23:00:00,537.21,6.45,1240.69,14.89,4474.36,53.69,True,True,False,False,True,True,False,5.5
1205,"Пельмения, ресторан",4.7,5348552841099386,ресторан,Пельмения,5348660212689578,25,набережная реки Фонтанки,"набережная реки Фонтанки, 25",Санкт-Петербург,Санкт-Петербург,Центральный район,Дворцовый,POINT(30.342986443162022 59.934200375224421),59.934202,30.342969,283.0,417.0,NaN,NaN,NaN,1000.0,Наличный расчёт; Оплата картой; Оплата по QR-коду,Русская кухня; Узбекская кухня; Грузинская кухня,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,True,Манты; Начос; Супы; Пельмени; Гёдза; Хинкали; ...,NaN,NaN,NaN,NaN,11:00:00,23:00:00,11:00:00,23:00:00,11:00:00,23:00:00,11:00:00,23:00:00,11:00:00,23:00:00,11:00:00,23:00:00,11:00:00,23:00:00,1629.88,19.56,2363.68,28.36,3761.06,45.13,True,True,True,False,True,False,False,5.1
308,"Империал, кафе-ресторан",4.6,5348552841707409,кафе-ресторан,Империал,5348660212798228,1,Смоленская,"Смоленская, 1",Санкт-Петербург,Санкт-Петербург,Московский район,Московская застава,POINT(30.319754482778709 59.903807810605976),59.903799,30.319758,148.0,179.0,NaN,NaN,True,1300.0,Оплата через банк; Оплата по QR-коду; Наличный...,Европейская кухня; Японская кухня; Кавказская ...,1.0,1.0,True,420.0,True,True,True,True,True,Паста; Гриль-бары; Десерты; Шашлык; Люля-кебаб...,NaN,NaN,NaN,NaN,11:00:00,00:00:00,11:00:00,00:00:00,11:00:00,00:00:00,11:00:00,00:00:00,11:00:00,00:00:00,12:00:00,00:00:00,12:00:00,00:00:00,4916.75,59.00,5287.76,63.45,320.64,3.85,True,True,False,False,True,True,False,4.9
